In [78]:
from random import shuffle
import glob,h5py, cv2, scipy, pickle, itertools, re, gc
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm
subj = 'blake'
filename = 'Datasets/'+'final_'+subj+'_RD-RA2_128_last.pkl'
print('File to be created: ' + filename)

File to be created: Datasets/final_blake_RD-RA2_128_last.pkl


In [79]:
# load labels
# subjects = ['10 dec emre mahbub', '12 jan emre', '11 jan mahbub emre ladi', '12 jan ozgur mahbub ladi emre', 
#             '13 jan ridvan ademola sean akthar emre', '14 jan ashwanth zeki', '15 apr ozgur',
#            '15 jan emin', '19 apr sean', '19 apr sevgi', '21 apr emin', '22 apr ladi', '22 apr sean',
#            '23 apr alperen', '23 apr mahbub', '5 apr sevgi']
# subjects = ['6 apr blake', '15 apr darrin', '16 apr devon asl2', '20 apr caroline', '21 apr blake']
subjects = ['6 apr blake', '21 apr blake']
mainpath = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/'
label_files = []
for i in range(len(subjects)):
    addr = glob.glob(mainpath+subjects[i]+'/labels/rangeDoppler/'+'*.txt')
    label_files.append(addr)
label_files = sum(label_files, [])
print('Num. of label files: '+str(len(label_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+label_files[i])

Num. of label files: 144
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/rangeDoppler/11040003_1617742238_5.txt
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/rangeDoppler/11030003_1617742086_3.txt
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/rangeDoppler/11020003_1617741930_5.txt
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/rangeDoppler/11040003_1617742238_1.txt
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/labels/rangeDoppler/11040003_1617742238_4.txt


In [80]:
# load videos
RD_files = []
for i in range(len(label_files)):
    addr = label_files[i].replace('labels/','').replace('txt','avi')
    RD_files.append(addr)
# RD_files = sum(RD_files, [])
print('Num. of label files: '+str(len(RD_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+RD_files[i])

Num. of label files: 144
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDoppler/11040003_1617742238_5.avi
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDoppler/11030003_1617742086_3.avi
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDoppler/11020003_1617741930_5.avi
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDoppler/11040003_1617742238_1.avi
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDoppler/11040003_1617742238_4.avi


In [81]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return np.array(dataframe.values)

In [82]:
# labels = []
# common = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/common labels/rangeDoppler/'
# cnt = []
# for i in tqdm(range(len(RD_files))):
#     seq = RD_files[i][-22]
#     it = RD_files[i][-5]
    
#     if seq == '1' and it == '1':
#         labels.append(np.squeeze(load_file(common+'1_1.txt')))
#         cnt.append(0)
#     elif seq == '1' and it != '1':
#         labels.append(np.squeeze(load_file(common+'1_2to5.txt')))
#         cnt.append(1)
#     elif seq == '2' and it == '1':
#         labels.append(np.squeeze(load_file(common+'2_1.txt')))
#         cnt.append(2)
#     elif seq == '2' and it != '1':
#         labels.append(np.squeeze(load_file(common+'2_2to5.txt')))
#         cnt.append(3)
#     elif seq == '3' and it == '1':
#         labels.append(np.squeeze(load_file(common+'3_1.txt')))
#         cnt.append(4)
#     elif seq == '3' and it != '1':
#         labels.append(np.squeeze(load_file(common+'3_2to5.txt')))
#         cnt.append(5)
#     elif seq == '4' and it == '1':
#         labels.append(np.squeeze(load_file(common+'4_1.txt')))
#         cnt.append(6)
#     elif seq == '4' and it != '1':
#         labels.append(np.squeeze(load_file(common+'4_2to5.txt')))
#         cnt.append(7)
#     elif seq == '5' and it == '1':
#         labels.append(np.squeeze(load_file(common+'5_1.txt')))
#         cnt.append(8)
#     elif seq == '5' and it != '1':
#         labels.append(np.squeeze(load_file(common+'5_2to5.txt')))
#         cnt.append(9)
# labels = np.array(labels)
# print(labels.shape)
# print(Counter(cnt))

In [83]:
RA_files = []
for i in range(len(RD_files)):
    addr = RD_files[i].replace('Doppler/','DOA/')
    RA_files.append(addr)
    
print('Num. of label files: '+str(len(RA_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+RA_files[i])

Num. of label files: 144
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDOA/11040003_1617742238_5.avi
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDOA/11030003_1617742086_3.avi
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDOA/11020003_1617741930_5.avi
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDOA/11040003_1617742238_1.avi
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/6 apr blake/rangeDOA/11040003_1617742238_4.avi


In [84]:
train_RD, test_RD, train_RA, test_RA = train_test_split(RD_files, RA_files, test_size=0.2, random_state=1)

train_RD = train_RD[0:800]
train_RA = train_RA[0:800]
test_RD = test_RD[0:200]
test_RA = test_RA[0:200]

In [85]:
def FrameCapture(path): 
      
    # Path to video file 
    vidObj = cv2.VideoCapture(path) 
    video_length = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
#     print ("Number of frames: ", video_length)
    # Used as counter variable 
    count = 0
    success, frame = vidObj.read() 
    # checks whether frames were extracted 
#     success = 1
    frames = []
    while success: 
  
        resized = cv2.resize(frame, (width,height), interpolation = cv2.INTER_LINEAR)
        success, frame = vidObj.read() 
#         print('Read a new frame: ', success)
        frames.append(resized) # .resized(width,height)
        count += 1
        if (count > (video_length-1)):
            break
    result = np.array([frames[i] for i in range(len(frames))])
    return result

In [86]:
def pad_video(video):
    if video.shape[0] < num_frames:
        cnt = 1
        numpad = num_frames-video.shape[0]
        video_padded = np.concatenate((video, np.zeros((numpad,video.shape[1],video.shape[2],video.shape[3]))), axis = 0)
#         while video_padded.shape[0] < num_frames:
#             video_padded = np.concatenate((video, video[-(num_frames-len(video)):]), axis = 0)
#             print('Padding '+str(cnt)+'/'+str(num_frames-video.shape[0]))
#             cnt += 1
    else:
        video_padded = video[:num_frames]
#     video_padded = video[:-1]
    return video_padded

In [87]:
width, height, channels = 128, 128, 3 
num_frames = 605

In [88]:
data = []
# data.append(y_train)
# data.append(y_test)
del RA_files, RD_files
gc.collect()

300

In [89]:
# from __future__ import print_function  # for Python2
# import sys

# local_vars = list(locals().items())
# for var, obj in local_vars:
#     print(var, sys.getsizeof(obj))

In [90]:
dur = 24 # sec
win = 0.2 # win len
n_step = int(dur/win)
des_w = width*n_step
des_h = height
print('num steps: '+str(n_step))
print('desired width: '+str(des_w))

num steps: 120
desired width: 15360


In [91]:
# train
videos = []
cnt = 1
for video in train_RD:
    clear_output(wait=True)
    print('Loading train video '+str(cnt)+'/'+str(len(train_RD))+ ' '+video)#,end="\r")
    vid = pad_video(FrameCapture(video))
#     vid = pad_video(FrameCapture(video))[0:600:5]
    videos.append(vid)
    cnt += 1
del train_RD
gc.collect()
data.append(np.array(videos)/255.)
del videos
gc.collect()

videos = []
cnt = 1
for video in train_RA:
    clear_output(wait=True)
    print('Loading train video '+str(cnt)+'/'+str(len(train_RA))+ ' '+video)#,end="\r")
#     video_di = pad_video(FrameCapture(video))
#     vid = np.reshape(pad_video(FrameCapture(video))[0:600],(n_step, 5, des_h, width, 3))
    vid = pad_video(FrameCapture(video))
#     vid = pad_video(FrameCapture(video))[0:600:5]
    videos.append(vid)
    cnt += 1
del train_RA
gc.collect()
data.append(np.array(videos)/255.)
del videos
gc.collect()

# test
videos = []
cnt = 1
for video in test_RD:
    clear_output(wait=True)
    print('Loading test video '+str(cnt)+'/'+str(len(test_RD))+ ' '+video)#,end="\r")
#     video_di = pad_video(FrameCapture(video))
#     vid = np.reshape(pad_video(FrameCapture(video))[0:600],(n_step, 5, des_h, width, 3))
#     vid = pad_video(FrameCapture(video))[0:600:5]
    vid = pad_video(FrameCapture(video))
    videos.append(vid)
    cnt += 1
del test_RD
gc.collect()
data.append(np.array(videos)/255.)
del videos
gc.collect()

videos = []
cnt = 1
for video in test_RA:
    clear_output(wait=True)
    print('Loading test video '+str(cnt)+'/'+str(len(test_RA))+ ' '+video)#,end="\r")
#     video_di = pad_video(FrameCapture(video))
#     vid = np.reshape(pad_video(FrameCapture(video))[0:600],(n_step, 5, des_h, width, 3))
#     vid = pad_video(FrameCapture(video))[0:600:5]
    vid = pad_video(FrameCapture(video))
    videos.append(vid)
    cnt += 1
del test_RA
gc.collect()
data.append(np.array(videos)/255.)
del videos
gc.collect()

Loading test video 29/29 /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/21 apr blake/rangeDOA/11040001_1619038244_3.avi


0

In [92]:
# # test
# videos = []
# cnt = 1
# test_RD = test_RD[0:150]
# for video in test_RD:
#     clear_output(wait=True)
#     print('Loading test video '+str(cnt)+'/'+str(len(test_RD))+ ' '+video)#,end="\r")
# #     video_di = pad_video(FrameCapture(video))
# #     vid = np.reshape(pad_video(FrameCapture(video))[0:600],(n_step, 5, des_h, width, 3))
# #     vid = pad_video(FrameCapture(video))[0:600:5]
#     vid = pad_video(FrameCapture(video))
#     videos.append(vid)
#     cnt += 1
# del test_RD
# gc.collect()
# data.append(np.array(videos)/255.)
# del videos
# gc.collect()

# videos = []
# cnt = 1
# test_RA = test_RA[0:150]
# for video in test_RA:
#     clear_output(wait=True)
#     print('Loading test video '+str(cnt)+'/'+str(len(test_RA))+ ' '+video)#,end="\r")
# #     video_di = pad_video(FrameCapture(video))
# #     vid = np.reshape(pad_video(FrameCapture(video))[0:600],(n_step, 5, des_h, width, 3))
# #     vid = pad_video(FrameCapture(video))[0:600:5]
#     vid = pad_video(FrameCapture(video))
#     videos.append(vid)
#     cnt += 1
# del test_RA
# gc.collect()
# data.append(np.array(videos)/255.)
# del videos
# gc.collect()

In [93]:
print(data[0].shape)
print(data[1].shape)
print(data[2].shape)
print(data[3].shape)
# print(data[4].shape)
# print(data[5].shape)

(115, 605, 128, 128, 3)
(115, 605, 128, 128, 3)
(29, 605, 128, 128, 3)
(29, 605, 128, 128, 3)


In [94]:
# # data = [train_RD_videos, test_RD_videos, y_train, y_test, train_RA_videos, test_RA_videos]
# with open(filename, 'wb') as output:  # Overwrites any existing file.
#         pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)
# print(filename+' created.')

In [95]:
f = h5py.File(filename[0:-3]+'hdf5', mode='w')
f.create_dataset("train_RD", data[0].shape, np.uint8)
f.create_dataset("test_RD", data[2].shape, np.uint8)  
f.create_dataset("train_RA", data[1].shape, np.uint8)
f.create_dataset("test_RA", data[3].shape, np.uint8)

f["train_RD"][...] = data[0]
f["test_RD"][...] = data[2]
f["train_RA"][...] = data[1]
f["test_RA"][...] = data[3]
f.close()
print(filename+' created.')

Datasets/final_blake_RD-RA2_128_last.pkl created.
